<a href="https://colab.research.google.com/github/Hsuyeon01/ESAA/blob/main/Surprise_(1121_%EA%B3%BC%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08. 파이썬 추천 시스템 패키지 - Surprise

## 1) Surprise 패키지 소개 

Surprise
- 파이썬 기반에서 사이킷런과 유사한 API와 프레임워크를 제공

In [ ]:
pip install scikit-surprise


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


주요 장점

- 다양한 추천 알고리즘, 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천시스템을 구축할 수 있음
- Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성됨


## 2) Surprise를 이용한 추천 시스템 구축

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
from surprise import Reader
import pandas as pd

df = pd.read_csv('ml-100k.csv')
reader = Reader(rating_scale=(1, 5))
col = ['user_id', 'item_id', 'rating']
df = df[col]
data = Dataset.load_from_df(df, reader)

In [ ]:
data = Dataset.load_builtin('ml-100k')

In [ ]:
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)


- 무비렌즈 사이트에서 내려받은 데이터 파일과 동일하게 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야 함
    - Surprise 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경

In [ ]:
algo = SVD()
algo.fit(trainset)

Surprise에서 추천을 예측하는 메서드는 test()와 predict()
- test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
- predict()는 개별 사용자와 영화에 대한 추천 평점을 반환
- test() 메서드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행할 결과

test()
- 개별 사용자 아이디(uid), 영화 아이디(iid), 실제 평점(r_ui) 정보에 기반해 Surprise의 추천 평점 데이터를 튜플 형태로 가지고 있음
- prediction 객체의 details 속성은 내부 처리 시 추천 예측을 알 수 없는 경우에 로그용 데이터를 남기는 데 사용
    - 'was_impossible=True'이면 예측값을 생성할 수 없는 데이터라는 의미

In [ ]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid=120, iid=282, r_ui=4.0, est=3.609312954284683, details={'was_impossible': False}),
 Prediction(uid=882, iid=291, r_ui=4.0, est=3.736489485395179, details={'was_impossible': False}),
 Prediction(uid=535, iid=507, r_ui=5.0, est=4.112606314096378, details={'was_impossible': False}),
 Prediction(uid=697, iid=244, r_ui=5.0, est=3.690272588209323, details={'was_impossible': False}),
 Prediction(uid=751, iid=385, r_ui=4.0, est=3.0962830507562984, details={'was_impossible': False})]

In [ ]:
# 리스트 객체 내에 내포된 Prediction 객체의 uid, iid, r_ui, est 등의 속성에 접근
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[(120, 282, 3.609312954284683),
 (882, 291, 3.736489485395179),
 (535, 507, 4.112606314096378)]

predict()
- 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est 형채로 반환

In [ ]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.53   {'was_impossible': False}


In [ ]:
# 추천 예측 평점과 실제 평점과의 차이 평가
accuracy.rmse(predictions)

RMSE: 0.9475


0.9474941921784467

## 3) Surprise 주요 모듈 소개

### Dataset

- Dataset.load_builtin(name='m1-100k') : 
    - 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 내려받음
    - m1-100k, ml-1M을 내려받을 수 있음 (디폴트는 m1-100k) 
    - 내려받은 데이터는 .surprise_data 디렉터리 밑에 저장되고, 해당 디렉터리에 데이터가 있으면 FTP에서 내려받지 않고 해당 데이터를 이용
- Dataset.load_from_file(file_path, reader) :
    - OS 파일에서 데이터를 로딩할 때 사용
    - 콤마, 탭 등에서 칼럼이 분리된 포맷의 OS 파일에서 데이터를 로딩
    - 입력 데이터로 OS 파일명, Reader로 파일의 포맷을 지정
- Dataset.load_from_df(df, reader) :
    - 판다스의 DataFrame에서 데이터를 로딩
    - 파라미터로 DataFrame을 입력받으며 DataFrame 역시 반드시 3개의 칼럼인 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼 순서가 정해져 있어야 함
 

### OS 파일 데이터를 Surprise 데이터 세트로 로딩

- 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안 된다는 것

In [ ]:
import pandas as pd
ratings = pd.read_csv('rating.csv')

In [ ]:
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤드를 모두 제거한 새로운 파일 생성
ratings.to_csv('ratings_noh.csv', index = False, header = False)

In [ ]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('ratings_noh.csv', reader = reader)

Reader 클래스의 주요 생성 파라미터 
- line_formal(string) : 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식
- sep(char) : 칼럼을 분리하는 분리자이며, 디폴트는 \t, 판다스는 DataFrame에서 입력받을 경우에는 기재
- rating_scale(tuple, optional) : 평점 값이 최소~최대 평점으로 설정, 디폴트는 (1,5)이지만 rating.csv 파일의 경우는 최소 평점이 0.5, 최대 평점이 5이므로 (0.5,5)로 설정

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25, 
                                    random_state=0)
# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)
# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7895


0.7895061187724683

### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('rating.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7895


0.7895061187724683

## 4) Surprise 추천 알고리즘

- SVD : 행렬 분해를 통한 잠재요인 협업 필터링을 위한 SVD 알고리즘
- KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- Baseline Only : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

Surprise SVD : 비용함수는 사용자 베이스라인의 편향성을 감안한 평점 예측에 Regularization을 적용한 것

SVD 클래스 입력 파라미터
- n_factors : 잠재 요인 K의 개수, 디폴트는 100, 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있음
- n_epochs : SGD 수행 시 반복횟수, 디폴트는 20
- biased : 베이슬인 사용자 편향 적용 여부, 디폴트는 True


## 5) 베이스라인 평점

베이스라인 평점 : 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부여하는 것

- 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
- 사용자 편향 점수 : 사용자별 아이템 평점 평균 값 - 전체 평균 평점
- 아이템 편향 점수 : 아이템별 평점 평균 값 - 전체 평균 평점

## 6) 교차 검증과 하이퍼 파라미터 튜닝

In [ ]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('rating.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

## 7) Surprise를 이용한 개인화 영화 추천 시스템 구축

- Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체를 변환하지 않으면 fit()을 통해 학습할 수 없음
-> 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 이용

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함
trainset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [ ]:
movies = pd.read_csv('movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점순으로 영화를 추천

In [ ]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값응로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    # 모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천 대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

In [ ]:
ef recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
   
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])